<a href="https://colab.research.google.com/github/cmadding/forge-of-games/blob/main/colab/alliance_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Alliance Manager** - Forge of Games

**README**

1.   [One time] Save this Colab notebook to your Google Drive (File > Save a copy in Drive). This will create a copy of this document and will open it in a new browser tab. Close this tab and proceed to your saved copy.
2.   Create new Google Spreadsheet document
3.   Update following properties in the `Configs` section below:
      *   `SPREADSHEET_ID`, `ALLIANCE_NAME`, `CUTOFF_DATE`, `GAME_WORLD_ID` must be set just **once**
      *   `ALLOW_ALLIANCE_ATH_LEADERBOARD_EXPORT` can be updated any time
4.    Open *Secrets* by clicking on a *key* icon in the left menu. Add 2 secrets: `HOH_USERNAME` and `HOH_PASSWORD`. Enter your real game login credentials as their values. Toggle *Notebook access* for both of them.
5.    **Important!** The script works by signing in to your game account and fetching required data. Then the data is sent to Forge of Games server for parsing. List of parsed player rankings is returned back to the script. Note, your game credentials are **never** sent to the Forge of Games. The server **does not** save or analyze data. However, you can allow sharing of treasure hunt alliance leadeboard by setting `ALLOW_ALLIANCE_ATH_LEADERBOARD_EXPORT` to `True` in the `Configs` section below. By doing so, you provide data for the *Treasure Hunt Ranking* section of the Alliance profile on Forge of Games. Even with this sharing enabled, the server **does not** analyze or share your internal player rankings.
6.    Click *Run all* button above. Colab environment will request access to your Google Drive in a new windows. Allow everything.
7.    While script is running, scroll to the bottom of the code. Notice the log messages popping up as the script is executed. Wait until you see *DONE*. Then, go to your spreadsheet document and check the results.
8.    When you need to update the data, simply click on *Run all* again. If you closed this notebook document, open it from your Google Drive.
9.    The tool is open-source. Contributions are welcome.
10.    If you find this tool useful, consider supporting Forge of Games project to benefit the whole game community. Details how to donate can be found here https://forgeofgames.com/support-us


In [ ]:
!pip install gspread oauth2client

**Imports**

In [ ]:
from google.colab import auth
from google.auth import default
from google.colab import userdata
from datetime import datetime
from gspread.utils import a1_range_to_grid_range, a1_to_rowcol, rowcol_to_a1
from dataclasses import dataclass
from typing import List
import string
import gspread
import requests
import re
import uuid
import base64

**Configs**

In [ ]:
# You can find the Spreadsheet ID in the URL of your Google Sheet. It's the long string of characters between '/d/' and '/edit'.
SPREADSHEET_ID = '1a3NWQu8-ivhjT13wD6gXq1vM8OdaQBTTiM_kVb-HnCE'

# Will become the name of the worksheet in your Spreadsheet document.
ALLIANCE_NAME = 'My Alliance'

# The earliest date from which events should be considered
# Normally, you would set it to the start date of the currently running ATH campaign
CUTOFF_DATE = datetime(2025, 10, 30)  # year, month, day

# 'un1' for main server or 'zz1' for beta server
GAME_WORLD_ID = 'un1'

# Change to True if you want to share treasure hunt alliance leaderboard with Forge of Games.
# Note, your internal players' treasure hunt points will NOT be shared.
ALLOW_ALLIANCE_ATH_LEADERBOARD_EXPORT = False

In [ ]:
FOG_API_URL_BASE = 'https://forgeofgames.com/api/hoh'
FOG_API_EVENTS = FOG_API_URL_BASE + '/inGameEvents/{world_id}/{event_definition_id}'
FOG_DATA_URL = f"https://forgeofgames-f.azurewebsites.net/api/hoh/inGameData/parse"

PROTOBUF_CONTENT_TYPE = 'application/x-protobuf'
JSON_CONTENT_TYPE = 'application/json'
GAME_LOGIN_URL = "https://{subdomain}.heroesgame.com/api/login"
ACCOUNT_PLAY_URL = "https://{subdomain}.heroesofhistorygame.com/core/api/account/play"
WAKEUP_API_URL = "https://{world_id}.heroesofhistorygame.com/game/wakeup"

PLAYER_ID_START_CELL = 'A3'
PLAYER_NAME_START_CELL = 'B3'
EVENT_ID_START_CELL = 'D1'

hoh_username = userdata.get('HOH_USERNAME')
hoh_password = userdata.get('HOH_PASSWORD')
if SPREADSHEET_ID == '':
  SPREADSHEET_ID = userdata.get('SPREADSHEET_ID')

In [ ]:
@dataclass
class PlayerAthPointsDto:
    eventId: str
    points: str
    playerId: int
    playerName: str

**Common API**

In [ ]:
def to_datetime(d: str):
  return datetime.fromisoformat(d)

def print_warning(msg: str):
  print(f"\033[38;5;208mWarning: {msg}\033[0m")

**Spreadsheet API**

In [ ]:
def authenticate_to_google():
  auth.authenticate_user()
  creds, _ = default()
  return gspread.authorize(creds)

In [ ]:
def get_next_cell_in_column(cell_coord: str):
  coord = parse_cell_coordinate(cell_coord)
  next_row_number = coord[1] + 1
  next_cell_coord = f"{coord[0]}{next_row_number}"

  return next_cell_coord

In [ ]:
def find_cell_in_row(worksheet: gspread.Worksheet, start_coord: str, value) -> tuple[gspread.Cell, bool]:
  coord = parse_cell_coordinate(start_coord)
  end_coord = rowcol_to_a1(coord[1], 100)
  row_cells = worksheet.range(f'{start_coord}:{end_coord}')

  for c in row_cells:
    if c.value == str(value):
      return (c, True)
    elif c.value == '':
      return (c, False)


In [ ]:
def find_cell_in_col(worksheet: gspread.Worksheet, start_coord: str, value) -> tuple[gspread.Cell, bool]:
  coord = parse_cell_coordinate(start_coord)
  end_coord = f'{coord[0]}100'
  col_cells = worksheet.range(f'{start_coord}:{end_coord}')

  for c in col_cells:
    if c.value == str(value):
      return (c, True)
    elif c.value == '':
      return (c, False)

In [ ]:
def parse_cell_coordinate(cell_coord: str) -> tuple[str, int] | None:
  match = re.match(r"([A-Z]+)(\d+)", cell_coord.upper())
  if not match:
    print(f"Invalid cell coordinate format: {cell_coord}")
    return None

  column_letter = match.group(1)
  row_number = int(match.group(2))

  return (column_letter, row_number)

In [ ]:
def autosize_col(worksheet: gspread.Worksheet, col: int):
  worksheet.spreadsheet.batch_update({
      "requests": [
          {
              "autoResizeDimensions": {
                  "dimensions": {
                      "sheetId": worksheet.id,
                      "dimension": "COLUMNS",
                      "startIndex": col - 1,
                      "endIndex": col
                  }
              }
          }
      ]
  })


**Innogames API**

In [ ]:
def default_headers():
    return {"Content-Type": JSON_CONTENT_TYPE}

def bin_data_headers(session_data):
    return {
        "X-AUTH-TOKEN": session_data["sessionId"],
        "X-Request-Id": str(uuid.uuid4()),
        "X-Platform": "browser",
        "X-ClientVersion": session_data["clientVersion"],
        "Accept-Encoding": "gzip",
        "Content-Type": PROTOBUF_CONTENT_TYPE,
        "Accept": PROTOBUF_CONTENT_TYPE,
    }

def login():
    session = requests.Session()

    payload = {
        "username": hoh_username,
        "password": hoh_password,
        "useRememberMe": False
    }

    subdomain = 'www'
    if GAME_WORLD_ID.startswith('zz'):
      subdomain = 'beta'
    response = session.post(GAME_LOGIN_URL.format(subdomain = subdomain), headers=default_headers(), json=payload)
    response.raise_for_status()
    login_data = response.json()

    redirect_res = session.get(login_data["redirectUrl"])
    redirect_res.raise_for_status()

    client_version_match = re.search(r'const\s+clientVersion\s*=\s*"([^"]+)"', redirect_res.text)
    if not client_version_match:
        raise Exception("Client version not found.")

    client_version = client_version_match.group(1)

    play_payload = {
        "createDeviceToken": False,
        "meta": {
            "clientVersion": client_version,
            "device": "browser",
            "deviceHardware": "browser",
            "deviceManufacturer": "none",
            "deviceName": "browser",
            "locale": "en_DK",
            "networkType": "wlan",
            "operatingSystemName": "browser",
            "operatingSystemVersion": "1",
            "userAgent": "hoh-player-ath-processor"
        },
        "network": "BROWSER_SESSION",
        "token": "",
        "worldId": None
    }

    subdomain = 'un0'
    if GAME_WORLD_ID.startswith('zz'):
      subdomain = 'zz0'
    res = session.post(ACCOUNT_PLAY_URL.format(subdomain = subdomain), headers=default_headers(), json=play_payload)
    res.raise_for_status()

    session_data = res.json()
    session_data["clientVersion"] = client_version
    return session_data


def get_bin_data(url, session_data):
    res = requests.post(url, headers=bin_data_headers(session_data))
    res.raise_for_status()
    return res.content

def get_game_data():
  session_data = login()
  bin_data = get_bin_data(WAKEUP_API_URL.format(world_id = GAME_WORLD_ID), session_data)
  return base64.b64encode(bin_data).decode('utf-8')

**Treasure Hunt API**

In [ ]:
def get_or_create_worksheet(gc):
  spreadsheet = gc.open_by_key(SPREADSHEET_ID)
  print(f"Successfully opened sheet: {spreadsheet.title}")
  try:
    worksheet = spreadsheet.worksheet(ALLIANCE_NAME)
    print(f"Worksheet '{ALLIANCE_NAME}' already exists.")
    return worksheet
  except gspread.WorksheetNotFound:
    print(f"Worksheet '{ALLIANCE_NAME}' not found. Creating it...")
    worksheet = spreadsheet.add_worksheet(title=ALLIANCE_NAME, rows="100", cols="100")
    create_worksheet_base_content(worksheet)
    print(f"Worksheet '{ALLIANCE_NAME}' created successfully.")
    return worksheet

In [ ]:
def create_worksheet_base_content(worksheet: gspread.Worksheet):
  worksheet.update_acell('C1', 'Event ID')
  worksheet.update_acell('A2', 'Player ID')
  worksheet.update_acell('B2', 'Player name')
  worksheet.update_acell('C2', 'Event dates')

  center_format = {
      "horizontalAlignment": "CENTER",
      "verticalAlignment": "MIDDLE"
  }

  worksheet.format('1:1', center_format)
  worksheet.format('2:2', center_format)

  grid_range = a1_range_to_grid_range('A1:C2', sheet_id=worksheet.id)
  worksheet.spreadsheet.batch_update({
      "requests": [
          {
              "updateSheetProperties": {
                    "properties": {
                        "sheetId": worksheet.id,
                        "gridProperties": {
                            "frozenRowCount": 2
                        }
                    },
                    "fields": "gridProperties.frozenRowCount"
                }
          },
          {
              "updateBorders": {
                "range": grid_range,
                "top":    {"style": "SOLID", "width": 1, "color": {"red": 0, "green": 0, "blue": 0}},
                "bottom": {"style": "SOLID", "width": 1, "color": {"red": 0, "green": 0, "blue": 0}},
                "left":   {"style": "SOLID", "width": 1, "color": {"red": 0, "green": 0, "blue": 0}},
                "right":  {"style": "SOLID", "width": 1, "color": {"red": 0, "green": 0, "blue": 0}},
            }
          }
      ]
  })

In [ ]:
def update_events(worksheet: gspread.Worksheet, events):
  sorted_events = sorted(events, key=lambda x: to_datetime(x['startAt']))

  for event in sorted_events:
    start_date_str = event['startAt']
    start_date = to_datetime(start_date_str)
    if(start_date < CUTOFF_DATE):
      continue

    event_id = event['id']
    cell_t = find_cell_in_row(worksheet, EVENT_ID_START_CELL, event_id)
    if not cell_t[1]:
      cell = cell_t[0]
      worksheet.update_cell(cell.row, cell.col, event_id)
      label_cell_coord = get_next_cell_in_column(cell.address)
      end_date_str = event['endAt']
      end_date = to_datetime(end_date_str)
      formatted_start_date = start_date.strftime('%d/%m/%y')
      formatted_end_date = end_date.strftime('%d/%m/%y')
      label = f"{formatted_start_date} - {formatted_end_date}"
      worksheet.update_acell(label_cell_coord, label)
      autosize_col(worksheet, cell.col)


In [ ]:
def update_players(worksheet: gspread.Worksheet, player_rankings: List[PlayerAthPointsDto]):
  _, player_name_column = a1_to_rowcol(PLAYER_NAME_START_CELL)
  sorted_player_rankings = sorted(player_rankings, key=lambda x: x.playerName)
  for pr in sorted_player_rankings:
    player_cell, found = find_cell_in_col(worksheet, PLAYER_ID_START_CELL, pr.playerId)
    if found:
      worksheet.update_cell(player_cell.row, player_name_column, pr.playerName)
    else:
      worksheet.update_cell(player_cell.row, player_cell.col, pr.playerId)
      worksheet.update_cell(player_cell.row, player_name_column, pr.playerName)



In [ ]:
def update_rankings(worksheet: gspread.Worksheet, player_rankings: List[PlayerAthPointsDto]):
  _, player_name_column = a1_to_rowcol(PLAYER_NAME_START_CELL)
  for pr in player_rankings:
    event_cell, event_cell_found = find_cell_in_row(worksheet, EVENT_ID_START_CELL, pr.eventId)
    if event_cell_found:
      player_cell, player_cell_found = find_cell_in_col(worksheet, PLAYER_ID_START_CELL, pr.playerId)
      if player_cell_found:
        worksheet.update_cell(player_cell.row, event_cell.col, pr.points)
      else:
        print_warning(f"Could not find player cell for Player - {pr.playerId}:{pr.playerName}")
    else:
      print_warning(f"Could not find event cell for EventId {pr.eventId}; Player - {pr.playerId}:{pr.playerName}")




**Forge of Games API**

In [ ]:
def get_ath_events():
  url = FOG_API_EVENTS.format(world_id=GAME_WORLD_ID, event_definition_id='TreasureHuntLeague')
  res = requests.get(url)
  res.raise_for_status()
  return res.json()


In [ ]:
def send_data(data: str):
  collectionCategoryIds = ['alliance']
  if ALLOW_ALLIANCE_ATH_LEADERBOARD_EXPORT:
    collectionCategoryIds.append('leaderboards')
  payload = {
      "base64ResponseData": data,
      "responseUrl": WAKEUP_API_URL.format(world_id = GAME_WORLD_ID),
      "collectionCategoryIds": collectionCategoryIds
  }
  res = requests.post(FOG_DATA_URL, headers=default_headers(), json=payload)
  res.raise_for_status()
  return res.json()


**MAIN**

In [ ]:
gc = authenticate_to_google()
ws = get_or_create_worksheet(gc)
events = get_ath_events()
print('Event list fetched.')
update_events(ws, events)
print('Event list updated.')
data = get_game_data()
print('Game data fetched.')
parsed_data = send_data(data)
print('Game data parsed.')
player_rankings = [PlayerAthPointsDto(**item_dict) for item_dict in parsed_data]
update_players(ws, player_rankings)
print('Players updated.')
update_rankings(ws, player_rankings)
print('Rankings updated.')

print()
print('DONE')